# OPTIMADE and *pymatgen*

# What is *pymatgen*?

[*pymatgen*](https://pymatgen.org) is a materials science analysis code written in the Python programming language. It helps power the [Materials Project](https://materialsproject.org)'s high-throughput DFT workflows. It supports integration with a wide variety of simulation codes and can perform many analysis tasks such as the generation of phase diagrams or diffraction patterns.

# The motivation behind this tutorial

**This tutorial is aimed either at:**

* People who are already familiar with using *pymatgen* or the Materials Project
    * In particular, anyone already using the Materials Project API through the `MPRester`, and who would like to start using the OPTIMADE API in a similar way

* People who like using Python and think they might appreciate an interface like the one provided by *pymatgen*.
    * *pymatgen* provides a lot of input/output routines (such as conversion to CIF, POSCAR, etc.) and analysis tools (such as determination of symmetry, analysis of possible bonds, etc.) that can be performed directly on structures retrieved from OPTIMADE providers.

**What this tutorial is not:**

* This is not necessarily the way everyone should be accessing OPTIMADE providers!
    * This tool may be useful to you, or it may not be. There are a lot of good tools available in our community. You are encouraged to try out different tools and find the one that's most useful for your own work.

* It is not currently the best way to access OPTIMADE APIs for advanced users.
    * It is still under development.
    * It is unit tested against several OPTIMADE providers but **some do not work yet**.
    * It only currently supports information retrieval from `/v1/structures/` routes.

# Pre-requisites

This tutorial is aimed at people who already have a basic understanding of Python, including how to import modules, the use of basic data structures like dictionaries and lists, and how to intantiate and use objects.

If you do not have this understanding of Python, this tutorial may help you become familiar, but you are highly encouraged to follow a dedicated Python course such as those provided by [Software Carpentry](https://software-carpentry.org).

# Install pymatgen

This tutorial uses the Python programming language. It can be run on any computer with Python installed. For convenience, here we are running in Google's "Colaboratory" notebook environment.

Before we begin, we must install the `pymatgen` package:

In [103]:
!pip install pymatgen pybtex retrying

Next, let us **verify the correct version of *pymatgen* is installed**. This is good practice to do before starting out! For this tutorial we need version 2022.0.14 or above. We also need the `pybtex` package installed.

In [104]:
from importlib_metadata import version

In [105]:
version("pymatgen")

'2022.0.14'

# Import and learn about the `OptimadeRester`

The `OptimadeRester` is a class that is designed to retrieve data from an OPTIMADE provider and automatically convert the data into *pymatgen* `Structure` objects. These `Structure` objects are designed as a good intermediate format for crystallographic structure analysis, transformation and input/output.

You can read documentation on the `OptimadeRester` here: https://pymatgen.org/pymatgen.ext.optimade.html

In [106]:
from pymatgen.ext.optimade import OptimadeRester

The first step is to inspect the **documentation** for the `OptimadeRester`. We can run:

In [107]:
OptimadeRester?

# Understanding "aliases" as shortcuts for accessing given providers

In [108]:
OptimadeRester.aliases

{'aflow': 'http://aflow.org/API/optimade/',
 'cod': 'https://www.crystallography.net/cod/optimade',
 'mcloud.2dstructures': 'https://aiida.materialscloud.org/2dstructures/optimade',
 'mcloud.2dtopo': 'https://aiida.materialscloud.org/2dtopo/optimade',
 'mcloud.curated-cofs': 'https://aiida.materialscloud.org/curated-cofs/optimade',
 'mcloud.li-ion-conductors': 'https://aiida.materialscloud.org/li-ion-conductors/optimade',
 'mcloud.optimade-sample': 'https://aiida.materialscloud.org/optimade-sample/optimade',
 'mcloud.pyrene-mofs': 'https://aiida.materialscloud.org/pyrene-mofs/optimade',
 'mcloud.scdm': 'https://aiida.materialscloud.org/autowannier/optimade',
 'mcloud.sssp': 'https://aiida.materialscloud.org/sssplibrary/optimade',
 'mcloud.stoceriaitf': 'https://aiida.materialscloud.org/stoceriaitf/optimade',
 'mcloud.tc-applicability': 'https://aiida.materialscloud.org/tc-applicability/optimade',
 'mcloud.threedd': 'https://aiida.materialscloud.org/3dd/optimade',
 'mp': 'https://optima

These aliases are useful since they can provide a quick shorthand for a given database without having to remember a full URL.

This list of aliases is updated periodically. However, new OPTIMADE providers can be made available and will be listed at https://providers.optimade.org. The `OptimadeRester` can query the OPTIMADE providers list to refresh the available aliases.

You can do this as follows, but be aware this might take a few moments:

In [109]:
opt = OptimadeRester()
opt.refresh_aliases()

Connecting to all known OPTIMADE providers, this will be slow. Please connect to only the OPTIMADE providers you want to query. Choose from: aflow, cod, mcloud.2dstructures, mcloud.2dtopo, mcloud.curated-cofs, mcloud.li-ion-conductors, mcloud.optimade-sample, mcloud.pyrene-mofs, mcloud.scdm, mcloud.sssp, mcloud.stoceriaitf, mcloud.tc-applicability, mcloud.threedd, mp, odbx, omdb.omdb_production, oqmd, tcod



odbx: 100%|██████████| 54/54 [00:19<00:00, 26.75it/s]

# Connecting to one or more OPTIMADE providers

Let's begin by connecting to the Materials Project (`mp`) and Materials Cloud "3DD" (`mcloud.threedd`) databases.

In [110]:
opt = OptimadeRester(["mp", "mcloud.threedd"])

We can find more information about the OPTIMADE providers we are connected to using the `describe()` method.

In [111]:
print(opt.describe())

OptimadeRester connected to:
Provider(name='The Materials Project', base_url='https://optimade.materialsproject.org', description='The Materials Project OPTIMADE endpoint', homepage='https://materialsproject.org', prefix='mp')
Provider(name='Materials Cloud', base_url='https://aiida.materialscloud.org/3dd/optimade', description='A platform for Open Science built for seamless sharing of resources in computational materials science', homepage='https://materialscloud.org', prefix='mcloud')


# Query for materials: binary nitrides case study

`OptimadeRester` provides an `get_structures` method. **It does not support all features of OPTIMADE filters** but is a good place to get started.

For this case study, we will search for materials containing nitrogen and that have two elements.

In [112]:
results = opt.get_structures(elements=["N"], nelements=2)




mp:   2%|▏         | 20/892 [00:00<?, ?it/s]


mp:   4%|▍         | 40/892 [00:00<00:10, 80.59it/s]


mp:   7%|▋         | 60/892 [00:00<00:16, 50.73it/s]


mp:   9%|▉         | 80/892 [00:01<00:15, 52.98it/s]


mp:  11%|█         | 100/892 [00:01<00:13, 57.95it/s]


mp:  13%|█▎        | 120/892 [00:01<00:15, 50.05it/s]


mp:  16%|█▌        | 140/892 [00:02<00:13, 54.44it/s]


mp:  18%|█▊        | 160/892 [00:02<00:13, 54.04it/s]


mp:  20%|██        | 180/892 [00:03<00:14, 47.85it/s]


mp:  22%|██▏       | 200/892 [00:03<00:14, 46.13it/s]


mp:  25%|██▍       | 220/892 [00:04<00:15, 44.63it/s]


mp:  27%|██▋       | 240/892 [00:04<00:14, 43.67it/s]


mp:  29%|██▉       | 260/892 [00:04<00:13, 46.71it/s]


mp:  31%|███▏      | 280/892 [00:05<00:12, 49.17it/s]


mp:  34%|███▎      | 300/892 [00:05<00:12, 46.48it/s]


mp:  36%|███▌      | 320/892 [00:06<00:10, 52.24it/s]


mp:  38%|███▊      | 340/892 [00:06<00:09, 56.87it/s]


mp:  40%|████      | 360/892 [00:06<00:08, 59.89it/s]


m

We see that the `OptimadeRester` does some of the hard work for us: it automatically retrieves multiple pages of results when many results are available, and also gives us a progress bar.

Let us inspect the `results`:

In [113]:
type(results)  # this method returns a dictionary, so let's examine the keys of this dictionary...

dict

In [114]:
results.keys()  # we see that the results dictionary is keyed by provider/alias

dict_keys(['mp', 'mcloud.threedd'])

In [115]:
results['mp'].keys()  # and these are then keyed by that database's unique identifier

dict_keys(['mp-1008610', 'mp-1019271', 'mp-1065394', 'mp-7790', 'mp-1015582', 'mp-1104441', 'mp-1019272', 'mp-1102235', 'mp-29973', 'mp-2245', 'mp-1014303', 'mp-1007824', 'mp-1245326', 'mp-973935', 'mp-834', 'mp-1018160', 'mp-1014298', 'mp-1214623', 'mp-1245013', 'mp-1244914', 'mp-27908', 'mp-32652', 'mp-1216472', 'mp-1014358', 'mp-1018733', 'mp-1009885', 'mp-1058019', 'mp-11660', 'mp-1209181', 'mp-510557', 'mp-1245132', 'mp-1009769', 'mvc-15478', 'mp-1865', 'mp-9981', 'mp-1077354', 'mp-10736', 'mp-1091417', 'mp-1245165', 'mp-1204550', 'mp-1245046', 'mp-661', 'mp-8101', 'mp-1001117', 'mp-2114', 'mp-1008833', 'mp-1008489', 'mp-1009483', 'mp-1188353', 'mp-1245244', 'mp-1077275', 'mp-1271276', 'mp-2828', 'mp-1078568', 'mp-1093992', 'mp-1213154', 'mp-2341', 'mvc-11235', 'mp-1014324', 'mp-1102114', 'mp-1244927', 'mp-634410', 'mp-567617', 'mp-1001826', 'mp-1002222', 'mp-1014995', 'mp-1072543', 'mp-1001916', 'mp-1064529', 'mp-754553', 'mp-1097011', 'mp-1096882', 'mp-1008921', 'mvc-13808', 'mp

So let us inspect one structure as an example:

In [116]:
example_structure = results['mp']['mp-804']
print(example_structure)

Full Formula (Ga2 N2)
Reduced Formula: GaN
abc   :   3.216290   3.216290   5.239962
angles:  90.000000  90.000000 120.000003
Sites (4)
  #  SP           a         b        c
---  ----  --------  --------  -------
  0  Ga    0.666667  0.333333  0.49912
  1  Ga    0.333333  0.666667  0.99912
  2  N     0.666667  0.333333  0.87588
  3  N     0.333333  0.666667  0.37588


We can then use *pymatgen* to further manipulate these `Structure` objects, for example to calculate the spacegroup or to convert to a CIF:

In [117]:
example_structure.get_space_group_info()

('P6_3mc', 186)

In [118]:
print(example_structure.to(fmt="cif", symprec=0.01))

# generated using pymatgen
data_GaN
_symmetry_space_group_name_H-M   P6_3mc
_cell_length_a   3.21629007
_cell_length_b   3.21629007
_cell_length_c   5.23996200
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   120.00000000
_symmetry_Int_Tables_number   186
_chemical_formula_structural   GaN
_chemical_formula_sum   'Ga2 N2'
_cell_volume   46.94282137
_cell_formula_units_Z   2
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
  2  'x-y, x, z+1/2'
  3  '-y, x-y, z'
  4  '-x, -y, z+1/2'
  5  '-x+y, -x, z'
  6  'y, -x+y, z+1/2'
  7  'y, x, z+1/2'
  8  'x, x-y, z'
  9  'x-y, -y, z+1/2'
  10  '-y, -x, z'
  11  '-x, -x+y, z+1/2'
  12  '-x+y, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Ga  Ga0  2  0.33333333  0.66666667  0.99912000  1
  N  N1  2  0.33333333  0.66666667  0.37588000  1



# Data analysis

This section I will use some code I prepared earlier to summarize the `results` into a tabular format (`DataFrame`).

In [119]:
import pandas as pd

In [120]:
records = []
for provider, structures in results.items():
    for identifier, structure in structures.items():
        records.append({
            "provider": provider,
            "identifier": identifier,
            "formula": structure.composition.reduced_formula,
            "spacegroup": structure.get_space_group_info()[0],
            "a_lattice_param": structure.lattice.a,
            "volume": structure.volume,
        })
df = pd.DataFrame(records)

In [121]:
df

,provider,identifier,formula,spacegroup,a_lattice_param,volume
0,mp,mp-1008610,AuN,Fm-3m,3.302053,25.458750
1,mp,mp-1019271,TaN2,P6/mmm,5.459583,72.277001
2,mp,mp-1065394,MoN,P6_3/mmc,2.870508,40.722201
3,mp,mp-7790,Ti2N,I4_1/amd,5.322762,76.524076
4,mp,mp-1015582,CrN2,Pbcn,4.405213,118.349470
...,...,...,...,...,...,...
974,mcloud.threedd,25786,N2O,Pa-3,5.842646,199.447562
975,mcloud.threedd,26371,SbN9,P-1,7.350390,322.370944
976,mcloud.threedd,26467,SmN,Fm-3m,3.559131,31.879910
977,mcloud.threedd,26627,CsN,Pm-3m,3.821708,55.817766


To pick one specific formula as an example, we can use tools from `pandas` to show the spacegroups present for that formula:

In [122]:
df[df["formula"] == "GaN"].spacegroup

11     P6_3/mmc
12           P1
18           P1
30           P1
38           P1
40           P1
49           P1
60           P1
128          P1
142      P6_3mc
147          P1
239      P6_3mc
246          P1
270          P1
271          P1
420          P1
471          P1
481          P1
493          P1
502          P1
503          P1
510          P1
515          P1
549          P1
571          P1
601          P1
621          P1
626       F-43m
633          P1
649          P1
673          P1
693          P1
708          P1
716       Fm-3m
722          P1
723          P1
731          P1
749          P1
801          P1
824          P1
855          P1
872          P1
881          P1
969      P6_3mc
Name: spacegroup, dtype: object

Here, we see that there are a few common high-symmetry spacegroups (such as $P6_3mc$) there are also many low-symmetry structures ($P1$).

I know that in this instance, this is because the $P1$ structures are actually amorphous and not crystalline. This highlights the importance of doing appropraiate **data cleaning** on retrieved data.

### Plotting data

As a quick example, we can also plot information in our table:

In [123]:
import plotly.express as px

In [124]:
px.bar(df, x="spacegroup", facet_row="provider")

**Remember, there is no single "best database" to use. Every database might be constructed for a specific purpose, subject to different biases, with different data qualities and sources.**

The ideal database for one scientist with one application in mind may be different to the ideal database for another scientist with a different application.

**The power of OPTIMADE is that you can query across multiple databases!**

# Advanced usage: querying using the OPTIMADE filter grammar

You can also query using an OPTIMADE filter as defined in the OPTIMADE specification and publication.

**This is recommended** for advanced queries to use the full power of OPTIMADE.

For example, the above query could have equally been performed as:

In [125]:
results = opt.get_structures_with_filter('(elements HAS ALL "N") AND (nelements=2)')


mp:  13%|█▎        | 120/892 [00:01<00:14, 54.45it/s]


mcloud.threedd: 100%|██████████| 87/87 [00:12<00:00,  5.91it/s]

mcloud.threedd: 100%|██████████| 87/87 [00:02<00:00, 31.86it/s]

# Advanced usage: retrieving provider-specific property information

The OPTIMADE specification allows for providers to include database-specific information in the returned data, prefixed by namespace.

To access this information with *pymatgen* we have to request "snls" (`StructureNL`) instead of "structures". A `StructureNL` is a `Structure` with additional metadata included, such as the URL it was downloaded from and any of this additional database-specific information.

In [126]:
results_snls = OptimadeRester("odbx").get_snls(nelements=2)



odbx:  37%|███▋      | 20/54 [00:00<?, ?it/s]

odbx:  74%|███████▍  | 40/54 [00:00<00:00, 31.84it/s]

odbx: 100%|██████████| 54/54 [00:01<00:00, 27.29it/s]

In [127]:
example_snl = results_snls['odbx']['odbx/2']

In [128]:
example_snl.data['_optimade']['_odbx_thermodynamics']

{'enthalpy': -816.9762466666666,
 'formation_energy': -0.39885422222209854,
 'hull_distance': 0.0004641666667737354,
 'relative_enthalpy': None,
 'total_energy': -816.9727734428334}

This extra data provided differs from every database, and sometimes from material to material, so some exploration is required!

# When Things Go Wrong and How to Get Help

Bugs may be present! The `OptimadeRester` is still fairly new.

If it does not work it is likely because of either:

* A bug in the *pymatgen* code. This may be reported directly to Matthew Horton at mkhorton@lbl.gov or an issue can be opened in the *pymatgen* code repository. Matt apologises in advance if this is the case! 

* An issue with a provider. This may be because the provider does not yet fully follow the OPTIMADE specification, because the provider is suffering an outage, or because the filters are not yet optimized with that provider.

    * If this happens, you may try to first increase the `timeout` value to something larger. The default is too low for some providers.

    * Otherwise, you may want to contact the provider directly, or create a post at the OPTIMADE discussion forum: https://matsci.org/optimade

# How to Get Involved

New developers are very welcome to add code to *pymatgen*! If you want to get involved, help fix bugs or add new features, your help would be very much appreciated. *pymatgen* can only exist and be what it is today thanks to the many efforts of its [development team](https://pymatgen.org/team.html).